# **ASSIGNMENT 2**

#**PART** **1**

**Question 1**: **computational graphs and gradients**

**Ans**.:Calling loss.backward() triggers PyTorch's automatic differentiation engine, Autograd.
As PyTorch build  computational graph dynamically as operations are performed  during forward pass. the backward call initiates a traversal backward through this graph, starting from the loss tensor itself. Using the chain rule of calculas,  AUTOGRAD efficiently calculates the gradient of the loss with respect to each tensor in the graph that has its requires_grad attribute set to True.
No, it doesnot overwrite the old gradient.

when we call loss.backward() , the newly computed gradients are added to whatever value is already present in the .grad() attribute of the parameters.
As they do not overwrite the previous values.

this acumulation is intentional and useful in certain scenarios, such as simulating larger batch sizes or training RNNs. However in a standard training loop where you process one batch at a time, we typically want to calculate the gradient of the current batch only.

This is why it is necessary to zero out the gradients at the beginning of each training iteration, usually before the forward pass or just before calling the loss.backward().
to do this we need an optimizer, optimizer.zero_grad().
<!-- after calculating the  gradient we need to use this to update models parameters, which is handled by the optimizers step method. -->

**Que 2**: **Tensors: view vs reshape**

In [ ]:
import torch

x = torch.arange(6)
y = x.view(2, 3)
print(y)

tensor([[0, 1, 2],
        [3, 4, 5]])


.view() is used when tensor is contiguous in memory. if it is not contiguous then we must call .contiguous() before .view().

In [ ]:
x = torch.arange(6)
y = x.reshape(2, 3)
print(y)

tensor([[0, 1, 2],
        [3, 4, 5]])


.reshape() can be used even if tensor is not contiguous.

if we are not sure about the memory stride then its is preferable to choose .reshape()

**Question 3 :**


**Ans :**

while peforming an operation in a tensor located on cpu and a tensor located on cuda:0, we will get runtime error, because all tensors need to be present on a same device. (The gpu memory can't directly access cpu memory).   to fix this we need to move one tensor to match the device of the other.

No, they do not automatically inherit the models's device. they must be manually assigned to the correct device.




model.to('cuda') does:
*   moves all model parameters (weights, biases) to the gpu.
*   doesn't affect new tensors created in forward().







**Que 4:** Tensor manipulation



Solution:

In [ ]:
import torch
images = torch.rand(4, 28, 28)
print(images)


tensor([[[1.5773e-02, 4.8750e-01, 3.5320e-01,  ..., 8.0440e-01,
          5.6666e-01, 5.8423e-01],
         [6.6705e-01, 6.4571e-01, 3.2476e-01,  ..., 8.6029e-01,
          6.5129e-01, 7.1146e-01],
         [5.3158e-01, 5.9746e-01, 5.9832e-02,  ..., 1.0987e-01,
          6.3854e-01, 1.3000e-01],
         ...,
         [2.5745e-01, 4.6752e-01, 4.9091e-01,  ..., 9.2901e-01,
          7.6525e-01, 3.3919e-01],
         [4.0543e-02, 7.3344e-01, 5.8648e-01,  ..., 1.9162e-01,
          4.0643e-01, 6.6662e-02],
         [8.6355e-02, 7.3089e-01, 5.4563e-01,  ..., 2.7718e-03,
          5.9419e-01, 3.4917e-01]],

        [[4.4576e-01, 1.8396e-01, 5.1665e-01,  ..., 1.2431e-02,
          8.0266e-01, 2.7098e-01],
         [6.3421e-01, 7.0266e-01, 8.5732e-01,  ..., 9.5510e-01,
          7.9920e-01, 1.2969e-01],
         [6.7174e-01, 5.0182e-01, 8.3014e-01,  ..., 3.8783e-01,
          3.2351e-01, 2.3745e-01],
         ...,
         [5.3634e-01, 4.1765e-01, 2.3927e-01,  ..., 6.3125e-01,
          5.737

In [ ]:
import torch

def binarize_images(images):
    return (images >= 0.5).float()


images = torch.rand(4, 28, 28)
binarized = binarize_images(images)
print(f"\nSample of original values:\n{images[0, :3, :3]}")
print(f"\nSample of binarized values:\n{binarized[0, :3, :3]}")


Sample of original values:
tensor([[0.0587, 0.0941, 0.6641],
        [0.7601, 0.6015, 0.2351],
        [0.8402, 0.7471, 0.1350]])

Sample of binarized values:
tensor([[0., 0., 1.],
        [1., 1., 0.],
        [1., 1., 0.]])


In [ ]:
x = torch.tensor(4.0, requires_grad = True)
y = x**3 + 2*x
y.backward()
dy_dx = 3 * x**2 + 2
if(dy_dx == x.grad):
  print(dy_dx)
  print(f"Gradient dy/dx at x={x.item()} is {x.grad}")
else:
  raise ValueError("calculated gradients do not match")



tensor(50., grad_fn=<AddBackward0>)
Gradient dy/dx at x=4.0 is 50.0


**Question 6:**

In [ ]:
from torch.utils.data import Dataset

class NumberDataset(Dataset):
    def __init__(self, numbers):
        self.numbers = numbers

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, idx):
        num = float(self.numbers[idx])
        input_tensor = torch.tensor(num, dtype = torch.float32)
        target_tensor = torch.tensor(num*2, dtype = torch.float32)

        return input_tensor, target_tensor





In [ ]:
ND = NumberDataset([1,2,3,4,5,874, 847, 877, 84, 84, 874, 74, 78, 874, 874, 874, 874, 8674, 763, 76743, 76764, 76764])
ND.__len__()
ND.__getitem__(3)

(tensor(4.), tensor(8.))

**Question 7:**

**Solution:**

In [ ]:
import torch.nn as nn
import torch.optim as optim
class SimpleClassifier(nn.Module):
  def __init__(self):
    super(SimpleClassifier, self).__init__()
    self.layer1 = nn.Linear(10,5)
    self.layer2 = nn.Linear(5,1)

  def forward(self, x):
     x = self.layer1(x)
     x = torch.relu(x)
     x = self.layer2(x)
     x = torch.sigmoid(x)
     return x


model = SimpleClassifier()
print(model)

SimpleClassifier(
  (layer1): Linear(in_features=10, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=1, bias=True)
)


In [ ]:
sample = torch.randn(1, 10)  # Batch size 1, input size 10
output = model(sample)
print(f"\nInput shape: {sample.shape}")
print(f"Output shape: {output.shape}")
print(f"Output value: {output.item():.4f}")
print(f"\nNumber of parameters: {sum(p.numel() for p in model.parameters())}")


Input shape: torch.Size([1, 10])
Output shape: torch.Size([1, 1])
Output value: 0.5166

Number of parameters: 61


In [ ]:
def train_step(model, inputs, targets, optimizer, criterion):
    optimizer.zero_grad()
    output = model(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()
    return loss.item()


In [ ]:
# Create optimizer (SGD with learning rate 0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Create loss function (Binary Cross Entropy Loss)
criterion = nn.BCELoss()

# Create dummy data
batch_size = 4
inputs = torch.randn(batch_size, 10)  # 4 samples, 10 features each
targets = torch.randint(0, 2, (batch_size, 1)).float()  # Binary targets (0 or 1)
print(inputs)
print(targets)

for step in range(10):
    loss_value = train_step(model, inputs, targets, optimizer, criterion)
    print(f"Step {step + 1:2d} | Loss: {loss_value:.6f}")

# Show predictions vs targets after training
with torch.no_grad():
    final_predictions = model(inputs)
    print(final_predictions)
    print("\nFinal Predictions vs Targets:")
    for i in range(batch_size):
        print(f"  Sample {i+1}: Pred = {final_predictions[i].item():.4f}, Target = {targets[i].item():.0f}")

tensor([[ 3.8004e-01,  5.5599e-01, -7.7038e-01, -6.4283e-01,  2.1728e+00,
          1.4544e-01,  1.6612e-01, -6.4306e-01, -2.6290e-01,  2.1560e-01],
        [ 2.1588e-01,  5.5082e-01, -4.5050e-01, -1.0578e+00,  8.0949e-01,
          1.3987e+00,  1.0361e+00, -1.3550e+00,  2.6925e-04,  2.9427e-02],
        [ 1.6844e+00,  1.1382e+00,  1.1477e+00,  1.1013e+00, -9.7366e-01,
         -1.2689e-02,  3.4809e-01, -5.3250e-01,  1.0044e-01, -1.0728e+00],
        [-2.7681e+00,  3.8551e-01,  2.2705e-01,  9.0160e-02, -1.1718e+00,
          2.6335e-01, -1.0512e+00, -3.7259e-01,  1.2236e+00,  8.5343e-01]])
tensor([[0.],
        [0.],
        [0.],
        [0.]])
Step  1 | Loss: 0.691049
Step  2 | Loss: 0.687328
Step  3 | Loss: 0.683632
Step  4 | Loss: 0.679963
Step  5 | Loss: 0.676320
Step  6 | Loss: 0.672702
Step  7 | Loss: 0.669109
Step  8 | Loss: 0.665541
Step  9 | Loss: 0.661998
Step 10 | Loss: 0.658480
tensor([[0.5178],
        [0.4482],
        [0.5149],
        [0.4359]])

Final Predictions vs T